In [ ]:
%pip install numpy pandas pillow tensorflow opencv-python matplotlib

In [ ]:
import os
import numpy as np
import tensorflow as tf
import cv2  
from PIL import Image
import matplotlib.pyplot as plt

# Define dataset paths
dataset_path = "/kaggle/input/moddelite/ModdeDataset"  # Original dataset
jpeg_dataset_path = "/kaggle/working/MODDE_JPEG"  # Path for converted JPEG images

# Step 1: Convert images to JPEG format
def convert_images_to_jpeg(src_dir, dest_dir):
    """ Converts all images in src_dir to JPEG format and saves them in dest_dir. """
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    for root, _, files in os.walk(src_dir):
        for file in files:
            src_file_path = os.path.join(root, file)  
            rel_path = os.path.relpath(root, src_dir)  
            dest_dir_path = os.path.join(dest_dir, rel_path)  
            dest_file_path = os.path.join(dest_dir_path, file)  

            if not os.path.exists(dest_dir_path):
                os.makedirs(dest_dir_path)

            # Skip files that already exist
            if os.path.exists(dest_file_path):
                continue

            try:
                # Convert to RGB JPEG format
                with Image.open(src_file_path) as img:
                    if img.mode != 'RGB':  
                        img = img.convert('RGB')
                    dest_file_path = os.path.splitext(dest_file_path)[0] + '.jpg'
                    img.save(dest_file_path, 'JPEG')
            except Exception as e:
                print(f"Failed to process {src_file_path}: {e}")

# Run conversion
convert_images_to_jpeg(dataset_path, jpeg_dataset_path)

# Step 2: Load dataset with preprocessing
def preprocess_image(image, label):
    """ Preprocesses the image: resize, normalize, and apply data augmentation. """
    # Resize to (256, 256)
    image = tf.image.resize(image, (256, 256))  

    # Normalize pixel values to [0,1]
    image = image / 255.0  

    # Data augmentation
    image = tf.image.random_flip_left_right(image)  # Random horizontal flip
    image = tf.image.random_brightness(image, max_delta=0.1)  # Adjust brightness
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)  # Adjust contrast

    return image, label

# Load dataset
data = tf.keras.utils.image_dataset_from_directory(
    jpeg_dataset_path,
    labels='inferred',  
    label_mode='categorical',  
    batch_size=32,  
    image_size=(256, 256),  
    shuffle=True,  
    seed=123  
)

# Apply preprocessing to dataset
data = data.map(preprocess_image)

# Step 3: Display sample images
plt.figure(figsize=(12, 8))
shown_classes = set()
grid_size = (3, 4)  

for images, labels in data.take(1):
    for i in range(len(images)):
        label_index = np.argmax(labels[i].numpy())  
        if label_index not in shown_classes:
            ax = plt.subplot(grid_size[0], grid_size[1], len(shown_classes) + 1)  
            ax.imshow(images[i].numpy())
            ax.set_title(data.class_names[label_index], fontsize=12, weight="bold", color="#333333")
            ax.axis("off")
            shown_classes.add(label_index)
        if len(shown_classes) == len(data.class_names):  
            break
    if len(shown_classes) == len(data.class_names):  
        break

plt.suptitle("Sample Images after Preprocessing", fontsize=16, weight="bold", color="#222222", y=0.92)
plt.tight_layout(pad=2.0, rect=[0, 0, 1, 0.92])
plt.show()
